In [44]:
import nltk

# If haven't done so, go and download
# nltk.download()

In [45]:
# import os
#
# trend_files = [f.replace('.json', '') for f in os.listdir('../dataset/trends')]

In [46]:
import pandas as pd

# tweet_df = pd.DataFrame([])

# for i in trend_files:
#     i = pd.read_json('../dataset/trends/'+i+'.json', orient='index')
#     tweet_df = tweet_df.append(i).reset_index().drop('index', axis=1)

# for combined dataset
# pd.read_json('../dataset/tweet-processed.json', orient='index')
tweets = pd.read_excel('../dataset/tweet-label-merged.xlsx').drop('Unnamed: 0', axis=1)[['emotion', 'raw_text']]
tweets.head()

,emotion,raw_text
0,NaN,REPLY AND RT WITH THE TAGS 💜\n\n#AMAsTNT BTS @...
1,NaN,Lockscreens|@BTS_twt \n#BTSxAMAs #AMAsTNT #BTS...
2,h,i love them 😭\n\n#AMAsTNT BTS @BTS_twt\n\nhttp...
3,NaN,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...
4,NaN,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...


In [47]:
tweets.shape

(13470, 2)

Goal: get sentiment. Therefore get only texts

In [48]:
# 1st: clear Twitter-specific terms
import re

def remove_twitter(text, for_gensim = False):
    twitter_stripped_text = re.sub('(@[A-Za-z0-9_]+)', '', text) if for_gensim else re.sub('(@)', '', text)   # Username removal
    twitter_stripped_text = re.sub(r'([http(s)?):\/\/(www\.)?a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*))', '', twitter_stripped_text)   # Link removal
    twitter_stripped_text = re.sub('(\#[A-Za-z0-9_]+)', '', twitter_stripped_text) if for_gensim else re.sub('(\#)', ' ', twitter_stripped_text)  # Hashtag removal, but preserving the word
    twitter_stripped_text = re.sub('([-&])', ' ', twitter_stripped_text)   # dash spacing, accommodating for better representation in spams
    return twitter_stripped_text

tweets['1st_clean_twitter'] = tweets['raw_text'].apply(lambda x: remove_twitter(x))
tweets['1st_clean_twitter_gensim'] = tweets['raw_text'].apply(lambda x: remove_twitter(x, for_gensim=True))
tweets.head()

,emotion,raw_text,1st_clean_twitter,1st_clean_twitter_gensim
0,NaN,REPLY AND RT WITH THE TAGS 💜\n\n#AMAsTNT BTS @...,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS
1,NaN,Lockscreens|@BTS_twt \n#BTSxAMAs #AMAsTNT #BTS...,Lockscreens|BTS_twt \n BTSxAMAs AMAsTNT BTS\...,Lockscreens| \n \nDon't repost! be honest!∅ \n
2,h,i love them 😭\n\n#AMAsTNT BTS @BTS_twt\n\nhttp...,i love them 😭\n\n AMAsTNT BTS BTS_twt\n\n,i love them 😭\n\n BTS \n\n
3,NaN,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...
4,NaN,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...


In [49]:
# 2nd: remove punctuations
import string
punctuation_gensim = re.sub('([.,?!])','', string.punctuation)

def remove_punctuation(text, for_gensim=False):
    clean_text = "".join([i for i in text if i not in punctuation_gensim]) if for_gensim else "".join([i for i in text if i not in string.punctuation])
    clean_text = re.sub('([’])', "'", clean_text)
    return clean_text

tweets['2nd_punctuation'] = tweets['1st_clean_twitter'].apply(lambda x: remove_punctuation(x))
tweets['2nd_punctuation_gensim'] = tweets['1st_clean_twitter_gensim'].apply(lambda x: remove_punctuation(x, for_gensim=True))
tweets.head()

,emotion,raw_text,1st_clean_twitter,1st_clean_twitter_gensim,2nd_punctuation,2nd_punctuation_gensim
0,NaN,REPLY AND RT WITH THE TAGS 💜\n\n#AMAsTNT BTS @...,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS
1,NaN,Lockscreens|@BTS_twt \n#BTSxAMAs #AMAsTNT #BTS...,Lockscreens|BTS_twt \n BTSxAMAs AMAsTNT BTS\...,Lockscreens| \n \nDon't repost! be honest!∅ \n,LockscreensBTStwt \n BTSxAMAs AMAsTNT BTS\nD...,Lockscreens \n \nDont repost! be honest!∅ \n
2,h,i love them 😭\n\n#AMAsTNT BTS @BTS_twt\n\nhttp...,i love them 😭\n\n AMAsTNT BTS BTS_twt\n\n,i love them 😭\n\n BTS \n\n,i love them 😭\n\n AMAsTNT BTS BTStwt\n\n,i love them 😭\n\n BTS \n\n
3,NaN,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,📸\n\nNew Kids on The Block bersama BTS\n\nDon...,📸\n\nNew Kids on The Block bersama BTS.\n\nDo...
4,NaN,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,📸\n\nShawn Stockman instagram update\n\n📎 \n...,📸\n\nShawn Stockman instagram update.\n\n📎 \...


In [50]:
import emoji

def remove_emoji(text):
    return re.sub(emoji.get_emoji_regexp(), r"", text)

tweets['3rd_emoji'] = tweets['2nd_punctuation'].apply(lambda x: remove_emoji(x))
tweets['3rd_emoji_gensim'] = tweets['2nd_punctuation_gensim'].apply(lambda x: remove_emoji(x))
tweets.head()

,emotion,raw_text,1st_clean_twitter,1st_clean_twitter_gensim,2nd_punctuation,2nd_punctuation_gensim,3rd_emoji,3rd_emoji_gensim
0,NaN,REPLY AND RT WITH THE TAGS 💜\n\n#AMAsTNT BTS @...,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS,REPLY AND RT WITH THE TAGS \n\n AMAsTNT BTS BT...,REPLY AND RT WITH THE TAGS \n\n BTS
1,NaN,Lockscreens|@BTS_twt \n#BTSxAMAs #AMAsTNT #BTS...,Lockscreens|BTS_twt \n BTSxAMAs AMAsTNT BTS\...,Lockscreens| \n \nDon't repost! be honest!∅ \n,LockscreensBTStwt \n BTSxAMAs AMAsTNT BTS\nD...,Lockscreens \n \nDont repost! be honest!∅ \n,LockscreensBTStwt \n BTSxAMAs AMAsTNT BTS\nD...,Lockscreens \n \nDont repost! be honest!∅ \n
2,h,i love them 😭\n\n#AMAsTNT BTS @BTS_twt\n\nhttp...,i love them 😭\n\n AMAsTNT BTS BTS_twt\n\n,i love them 😭\n\n BTS \n\n,i love them 😭\n\n AMAsTNT BTS BTStwt\n\n,i love them 😭\n\n BTS \n\n,i love them \n\n AMAsTNT BTS BTStwt\n\n,i love them \n\n BTS \n\n
3,NaN,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,📸\n\nNew Kids on The Block bersama BTS\n\nDon...,📸\n\nNew Kids on The Block bersama BTS.\n\nDo...,\n\nNew Kids on The Block bersama BTS\n\nDonn...,\n\nNew Kids on The Block bersama BTS.\n\nDon...
4,NaN,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,📸\n\nShawn Stockman instagram update\n\n📎 \n...,📸\n\nShawn Stockman instagram update.\n\n📎 \...,\n\nShawn Stockman instagram update\n\n \n\n...,\n\nShawn Stockman instagram update.\n\n \n\...


In [51]:
# tweets['4th_lower'] = tweets['3rd_emoji'].apply(nltk.tokenize.word_tokenize)
# tweets.head()
tweets['4th_lower'] = tweets['3rd_emoji'].str.lower()
tweets['4th_lower_gensim'] = tweets['3rd_emoji_gensim'].str.lower()
tweets.head()

,emotion,raw_text,1st_clean_twitter,1st_clean_twitter_gensim,2nd_punctuation,2nd_punctuation_gensim,3rd_emoji,3rd_emoji_gensim,4th_lower,4th_lower_gensim
0,NaN,REPLY AND RT WITH THE TAGS 💜\n\n#AMAsTNT BTS @...,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS,REPLY AND RT WITH THE TAGS \n\n AMAsTNT BTS BT...,REPLY AND RT WITH THE TAGS \n\n BTS,reply and rt with the tags \n\n amastnt bts bt...,reply and rt with the tags \n\n bts
1,NaN,Lockscreens|@BTS_twt \n#BTSxAMAs #AMAsTNT #BTS...,Lockscreens|BTS_twt \n BTSxAMAs AMAsTNT BTS\...,Lockscreens| \n \nDon't repost! be honest!∅ \n,LockscreensBTStwt \n BTSxAMAs AMAsTNT BTS\nD...,Lockscreens \n \nDont repost! be honest!∅ \n,LockscreensBTStwt \n BTSxAMAs AMAsTNT BTS\nD...,Lockscreens \n \nDont repost! be honest!∅ \n,lockscreensbtstwt \n btsxamas amastnt bts\nd...,lockscreens \n \ndont repost! be honest!∅ \n
2,h,i love them 😭\n\n#AMAsTNT BTS @BTS_twt\n\nhttp...,i love them 😭\n\n AMAsTNT BTS BTS_twt\n\n,i love them 😭\n\n BTS \n\n,i love them 😭\n\n AMAsTNT BTS BTStwt\n\n,i love them 😭\n\n BTS \n\n,i love them \n\n AMAsTNT BTS BTStwt\n\n,i love them \n\n BTS \n\n,i love them \n\n amastnt bts btstwt\n\n,i love them \n\n bts \n\n
3,NaN,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,📸\n\nNew Kids on The Block bersama BTS\n\nDon...,📸\n\nNew Kids on The Block bersama BTS.\n\nDo...,\n\nNew Kids on The Block bersama BTS\n\nDonn...,\n\nNew Kids on The Block bersama BTS.\n\nDon...,\n\nnew kids on the block bersama bts\n\ndonn...,\n\nnew kids on the block bersama bts.\n\ndon...
4,NaN,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,📸\n\nShawn Stockman instagram update\n\n📎 \n...,📸\n\nShawn Stockman instagram update.\n\n📎 \...,\n\nShawn Stockman instagram update\n\n \n\n...,\n\nShawn Stockman instagram update.\n\n \n\...,\n\nshawn stockman instagram update\n\n \n\n...,\n\nshawn stockman instagram update.\n\n \n\...


In [52]:
tweets['5th_tokenized'] = tweets['4th_lower'].apply(nltk.tokenize.word_tokenize)
tweets['5th_tokenized_gensim'] = tweets['4th_lower_gensim'].apply(nltk.tokenize.word_tokenize)
tweets.head()

,emotion,raw_text,1st_clean_twitter,1st_clean_twitter_gensim,2nd_punctuation,2nd_punctuation_gensim,3rd_emoji,3rd_emoji_gensim,4th_lower,4th_lower_gensim,5th_tokenized,5th_tokenized_gensim
0,NaN,REPLY AND RT WITH THE TAGS 💜\n\n#AMAsTNT BTS @...,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS,REPLY AND RT WITH THE TAGS \n\n AMAsTNT BTS BT...,REPLY AND RT WITH THE TAGS \n\n BTS,reply and rt with the tags \n\n amastnt bts bt...,reply and rt with the tags \n\n bts,"[reply, and, rt, with, the, tags, amastnt, bts...","[reply, and, rt, with, the, tags, bts]"
1,NaN,Lockscreens|@BTS_twt \n#BTSxAMAs #AMAsTNT #BTS...,Lockscreens|BTS_twt \n BTSxAMAs AMAsTNT BTS\...,Lockscreens| \n \nDon't repost! be honest!∅ \n,LockscreensBTStwt \n BTSxAMAs AMAsTNT BTS\nD...,Lockscreens \n \nDont repost! be honest!∅ \n,LockscreensBTStwt \n BTSxAMAs AMAsTNT BTS\nD...,Lockscreens \n \nDont repost! be honest!∅ \n,lockscreensbtstwt \n btsxamas amastnt bts\nd...,lockscreens \n \ndont repost! be honest!∅ \n,"[lockscreensbtstwt, btsxamas, amastnt, bts, do...","[lockscreens, dont, repost, !, be, honest, !, ∅]"
2,h,i love them 😭\n\n#AMAsTNT BTS @BTS_twt\n\nhttp...,i love them 😭\n\n AMAsTNT BTS BTS_twt\n\n,i love them 😭\n\n BTS \n\n,i love them 😭\n\n AMAsTNT BTS BTStwt\n\n,i love them 😭\n\n BTS \n\n,i love them \n\n AMAsTNT BTS BTStwt\n\n,i love them \n\n BTS \n\n,i love them \n\n amastnt bts btstwt\n\n,i love them \n\n bts \n\n,"[i, love, them, amastnt, bts, btstwt]","[i, love, them, bts]"
3,NaN,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,📸\n\nNew Kids on The Block bersama BTS\n\nDon...,📸\n\nNew Kids on The Block bersama BTS.\n\nDo...,\n\nNew Kids on The Block bersama BTS\n\nDonn...,\n\nNew Kids on The Block bersama BTS.\n\nDon...,\n\nnew kids on the block bersama bts\n\ndonn...,\n\nnew kids on the block bersama bts.\n\ndon...,"[new, kids, on, the, block, bersama, bts, donn...","[new, kids, on, the, block, bersama, bts, ., d..."
4,NaN,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,📸\n\nShawn Stockman instagram update\n\n📎 \n...,📸\n\nShawn Stockman instagram update.\n\n📎 \...,\n\nShawn Stockman instagram update\n\n \n\n...,\n\nShawn Stockman instagram update.\n\n \n\...,\n\nshawn stockman instagram update\n\n \n\n...,\n\nshawn stockman instagram update.\n\n \n\...,"[shawn, stockman, instagram, update, btsxamas,...","[shawn, stockman, instagram, update, ., 방탄소년단]"


In [53]:
# On the other project that I made I chose stopwords because I was going for bag-of-words approach.
crf = nltk.CRFTagger()
crf.set_model_file('../resources/pos_tag/all_indo_man_tag_corpus_model.crf.tagger')

_ = crf.tag_sents(tweets['5th_tokenized'])

In [54]:
_

[[('reply', 'FW'),
  ('and', 'FW'),
  ('rt', 'FW'),
  ('with', 'FW'),
  ('the', 'FW'),
  ('tags', 'FW'),
  ('amastnt', 'FW'),
  ('bts', 'FW'),
  ('btstwt', 'FW')],
 [('lockscreensbtstwt', 'FW'),
  ('btsxamas', 'FW'),
  ('amastnt', 'FW'),
  ('bts', 'FW'),
  ('dont', 'FW'),
  ('repost', 'FW'),
  ('be', 'FW'),
  ('honest∅', 'FW'),
  ('vica', 'FW')],
 [('i', 'FW'),
  ('love', 'FW'),
  ('them', 'FW'),
  ('amastnt', 'FW'),
  ('bts', 'FW'),
  ('btstwt', 'FW')],
 [('new', 'FW'),
  ('kids', 'FW'),
  ('on', 'FW'),
  ('the', 'FW'),
  ('block', 'FW'),
  ('bersama', 'IN'),
  ('bts', 'FW'),
  ('donnie', 'FW'),
  ('wahlberg', 'FW'),
  ('menggunakan', 'VB'),
  ('t', 'FW'),
  ('shirt', 'FW'),
  ('bts', 'FW'),
  ('love', 'FW'),
  ('yourself', 'FW'),
  ('world', 'FW'),
  ('tour', 'FW'),
  ('btsxamas', 'FW'),
  ('amastnt', 'FW'),
  ('방탄소년단', 'FW'),
  ('bts', 'FW'),
  ('btstwt', 'FW')],
 [('shawn', 'NN'),
  ('stockman', 'NN'),
  ('instagram', 'FW'),
  ('update', 'FW'),
  ('btsxamas', 'FW'),
  ('amastnt', '

From here we can see that the POS tagger we use did not tag a bit of the text correctly. Another solution that we have is to load an established Indonesian corpus (from Flair), train the model ourselves, and use it as our POS tagger.

In [55]:
from flair.models import SequenceTagger

pos_tag = SequenceTagger.load('../resources/taggers/indonesian-pos/final-model.pt')

2021-12-12 10:40:16,799 loading file ../resources/taggers/indonesian-pos/final-model.pt


In [56]:
from flair.data import Sentence

Sentence('Budi pergi ke pasar')
# pos_tag.predict(tweets['4th_tokenized'].iloc[0])

Sentence: "Budi pergi ke pasar"   [− Tokens: 4]

In [57]:
tweets['sentence'] = tweets['5th_tokenized'].apply(Sentence)
tweets['sentence'].head()

0    (Token: 1 reply, Token: 2 and, Token: 3 rt, To...
1    (Token: 1 lockscreensbtstwt, Token: 2 btsxamas...
2    (Token: 1 i, Token: 2 love, Token: 3 them, Tok...
3    (Token: 1 new, Token: 2 kids, Token: 3 on, Tok...
4    (Token: 1 shawn, Token: 2 stockman, Token: 3 i...
Name: sentence, dtype: object

In [58]:
tweets['sentence']

0        (Token: 1 reply, Token: 2 and, Token: 3 rt, To...
1        (Token: 1 lockscreensbtstwt, Token: 2 btsxamas...
2        (Token: 1 i, Token: 2 love, Token: 3 them, Tok...
3        (Token: 1 new, Token: 2 kids, Token: 3 on, Tok...
4        (Token: 1 shawn, Token: 2 stockman, Token: 3 i...
                               ...                        
13465    (Token: 1 iloveu, Token: 2 so, Token: 3 much, ...
13466    (Token: 1 iiii, Token: 2 mau, Token: 3 liat, T...
13467                (Token: 1 seokjin, Token: 2 sayangku)
13468    (Token: 1 seokjin, Token: 2 i, Token: 3 love, ...
13469    (Token: 1 surprise, Token: 2 nya, Token: 3 la,...
Name: sentence, Length: 13470, dtype: object

In [59]:
tweets['sentence'].apply(pos_tag.predict)

0        None
1        None
2        None
3        None
4        None
         ... 
13465    None
13466    None
13467    None
13468    None
13469    None
Name: sentence, Length: 13470, dtype: object

In [60]:
tweets['sentence']

0        (Token: 1 reply, Token: 2 and, Token: 3 rt, To...
1        (Token: 1 lockscreensbtstwt, Token: 2 btsxamas...
2        (Token: 1 i, Token: 2 love, Token: 3 them, Tok...
3        (Token: 1 new, Token: 2 kids, Token: 3 on, Tok...
4        (Token: 1 shawn, Token: 2 stockman, Token: 3 i...
                               ...                        
13465    (Token: 1 iloveu, Token: 2 so, Token: 3 much, ...
13466    (Token: 1 iiii, Token: 2 mau, Token: 3 liat, T...
13467                (Token: 1 seokjin, Token: 2 sayangku)
13468    (Token: 1 seokjin, Token: 2 i, Token: 3 love, ...
13469    (Token: 1 surprise, Token: 2 nya, Token: 3 la,...
Name: sentence, Length: 13470, dtype: object

In [61]:
_ = tweets['sentence'].iloc[1]
_.to_tagged_string()

'lockscreensbtstwt <PROPN> btsxamas <PROPN> amastnt <PROPN> bts <PROPN> dont <PROPN> repost <PROPN> be <PROPN> honest∅ <PROPN> vica <PROPN>'

In [62]:
re.findall(r'(<[^>]*>)', _.to_tagged_string())

['<PROPN>',
 '<PROPN>',
 '<PROPN>',
 '<PROPN>',
 '<PROPN>',
 '<PROPN>',
 '<PROPN>',
 '<PROPN>',
 '<PROPN>']

In [63]:
# Trial

re.sub('[ ]<', '_', _.to_tagged_string()).replace('>', '')

'lockscreensbtstwt_PROPN btsxamas_PROPN amastnt_PROPN bts_PROPN dont_PROPN repost_PROPN be_PROPN honest∅_PROPN vica_PROPN'

In [64]:
sentence_tagged = []

for i in tweets['sentence']:
    sentence_tagged.append(re.sub(r'[ ]<', '_', i.to_tagged_string()).replace('>', ''))

sentence_tagged

['reply_PROPN and_PROPN rt_PROPN with_PROPN the_PROPN tags_PROPN amastnt_PROPN bts_PROPN btstwt_PUNCT',
 'lockscreensbtstwt_PROPN btsxamas_PROPN amastnt_PROPN bts_PROPN dont_PROPN repost_PROPN be_PROPN honest∅_PROPN vica_PROPN',
 'i_PROPN love_PROPN them_PROPN amastnt_PROPN bts_PROPN btstwt_PUNCT',
 'new_PROPN kids_PROPN on_PROPN the_PROPN block_PROPN bersama_ADP bts_NOUN donnie_PROPN wahlberg_PROPN menggunakan_VERB t_NOUN shirt_NOUN bts_NOUN love_PROPN yourself_PROPN world_PROPN tour_PROPN btsxamas_PROPN amastnt_PROPN 방탄소년단_PROPN bts_PROPN btstwt_PUNCT',
 'shawn_PROPN stockman_PROPN instagram_PROPN update_PROPN btsxamas_PROPN amastnt_PROPN 방탄소년단_PROPN bts_PROPN btstwt_PUNCT',
 'jungkook_PROPN amas_PROPN hd_PROPN wallpaper_PROPN amp_PROPN lockscreen_PROPN reply_PROPN and_PROPN retweet_PROPN amastnt_PUNCT bts_NOUN btstwt_NOUN bts_NOUN btsxamas_PROPN 방탄소년단_PROPN btswallpaper_PROPN jungkook_PROPN btslockscreen_PROPN 방탄배경_PUNCT',
 'hobi_NOUN and_PROPN becky_PROPN amastnt_PROPN bts_PROPN bt

In [65]:
tweets['6_tagged'] = sentence_tagged
# tweets['6_tagged'] = tweets['6_tagged'].apply(nltk.tokenize.word_tokenize)

In [66]:
tweets.head()

,emotion,raw_text,1st_clean_twitter,1st_clean_twitter_gensim,2nd_punctuation,2nd_punctuation_gensim,3rd_emoji,3rd_emoji_gensim,4th_lower,4th_lower_gensim,5th_tokenized,5th_tokenized_gensim,sentence,6_tagged
0,NaN,REPLY AND RT WITH THE TAGS 💜\n\n#AMAsTNT BTS @...,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS,REPLY AND RT WITH THE TAGS 💜\n\n AMAsTNT BTS B...,REPLY AND RT WITH THE TAGS 💜\n\n BTS,REPLY AND RT WITH THE TAGS \n\n AMAsTNT BTS BT...,REPLY AND RT WITH THE TAGS \n\n BTS,reply and rt with the tags \n\n amastnt bts bt...,reply and rt with the tags \n\n bts,"[reply, and, rt, with, the, tags, amastnt, bts...","[reply, and, rt, with, the, tags, bts]","(Token: 1 reply, Token: 2 and, Token: 3 rt, To...",reply_PROPN and_PROPN rt_PROPN with_PROPN the_...
1,NaN,Lockscreens|@BTS_twt \n#BTSxAMAs #AMAsTNT #BTS...,Lockscreens|BTS_twt \n BTSxAMAs AMAsTNT BTS\...,Lockscreens| \n \nDon't repost! be honest!∅ \n,LockscreensBTStwt \n BTSxAMAs AMAsTNT BTS\nD...,Lockscreens \n \nDont repost! be honest!∅ \n,LockscreensBTStwt \n BTSxAMAs AMAsTNT BTS\nD...,Lockscreens \n \nDont repost! be honest!∅ \n,lockscreensbtstwt \n btsxamas amastnt bts\nd...,lockscreens \n \ndont repost! be honest!∅ \n,"[lockscreensbtstwt, btsxamas, amastnt, bts, do...","[lockscreens, dont, repost, !, be, honest, !, ∅]","(Token: 1 lockscreensbtstwt, Token: 2 btsxamas...",lockscreensbtstwt_PROPN btsxamas_PROPN amastnt...
2,h,i love them 😭\n\n#AMAsTNT BTS @BTS_twt\n\nhttp...,i love them 😭\n\n AMAsTNT BTS BTS_twt\n\n,i love them 😭\n\n BTS \n\n,i love them 😭\n\n AMAsTNT BTS BTStwt\n\n,i love them 😭\n\n BTS \n\n,i love them \n\n AMAsTNT BTS BTStwt\n\n,i love them \n\n BTS \n\n,i love them \n\n amastnt bts btstwt\n\n,i love them \n\n bts \n\n,"[i, love, them, amastnt, bts, btstwt]","[i, love, them, bts]","(Token: 1 i, Token: 2 love, Token: 3 them, Tok...",i_PROPN love_PROPN them_PROPN amastnt_PROPN bt...
3,NaN,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,[📸]\n\nNew Kids on The Block bersama BTS.\n\n...,📸\n\nNew Kids on The Block bersama BTS\n\nDon...,📸\n\nNew Kids on The Block bersama BTS.\n\nDo...,\n\nNew Kids on The Block bersama BTS\n\nDonn...,\n\nNew Kids on The Block bersama BTS.\n\nDon...,\n\nnew kids on the block bersama bts\n\ndonn...,\n\nnew kids on the block bersama bts.\n\ndon...,"[new, kids, on, the, block, bersama, bts, donn...","[new, kids, on, the, block, bersama, bts, ., d...","(Token: 1 new, Token: 2 kids, Token: 3 on, Tok...",new_PROPN kids_PROPN on_PROPN the_PROPN block_...
4,NaN,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,[📸]\n\nShawn Stockman instagram update.\n\n📎 :...,📸\n\nShawn Stockman instagram update\n\n📎 \n...,📸\n\nShawn Stockman instagram update.\n\n📎 \...,\n\nShawn Stockman instagram update\n\n \n\n...,\n\nShawn Stockman instagram update.\n\n \n\...,\n\nshawn stockman instagram update\n\n \n\n...,\n\nshawn stockman instagram update.\n\n \n\...,"[shawn, stockman, instagram, update, btsxamas,...","[shawn, stockman, instagram, update, ., 방탄소년단]","(Token: 1 shawn, Token: 2 stockman, Token: 3 i...",shawn_PROPN stockman_PROPN instagram_PROPN upd...


In [67]:
tweets.to_json('../dataset/tweet-processed.json', orient='index', default_handler=str)
tweets.to_excel('../dataset/tweet-label.xlsx')